In [58]:
### 라이브러리 정의하기
# 모델 또는 파일 저장 라이브러리
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
### 데이터 수집하기
# 훈련 및 테스트 데이터셋 각각 다운로드하기

### 훈련데이터 다운로드하기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                           filename = "ratings_train.txt")

('ratings_train.txt', <http.client.HTTPMessage at 0x27e65b63280>)

In [3]:
### 테스트 데이터 다운로드하기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                           filename = "ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x27e1d9467f0>)

In [59]:
### 훈련 및 테스트 데이터 읽어 들이기
train_data = pd.read_table("./ratings_train.txt")
test_data = pd.read_table("./ratings_test.txt")
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [60]:
### 훈련 데이터셋 100개만 가지고 진행...
train_data = train_data.iloc[ : 100, : ]
test_data = test_data.iloc[ : 100, : ]
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        100 non-null    int64 
 1   document  100 non-null    object
 2   label     100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        100 non-null    int64 
 1   document  100 non-null    object
 2   label     100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


In [61]:
### 데이터 정제하기
# 중복 데이터 확인하기
train_data["document"].nunique()

100

In [7]:
train_data["label"].nunique()

2

In [62]:
### 중복데이터 제거하기
## inplace = True : 별도의 변수에 넣지 않아도 바로 반영됨
train_data.drop_duplicates(subset=["document"], inplace=True)

In [64]:
len(train_data)

100

In [65]:
### label 데이터에서 긍정/부정의 데이터 빈도를 확인해주세요
train_data.groupby("label").size().reset_index(name="count")

,label,count
0,0,46
1,1,54


In [66]:
### 데이터 정제하기
# - 한글과 공백을 제외하고 모두 제거하기
train_data["document"] = \
    train_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
train_data

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
95,8763660,어내스트와 셀레스틴 완전 강추에요 정말 재밌습니다,1
96,9361974,재미있는영화입니다,1
97,7928957,클라라볼라고화신본거아닌데,0
98,10250221,진짜 보면서 너무 슬펐던 영화다,1


In [67]:
## (for문을 많이 돌려야되므로 데이터프레임에서 처리하기)
### 특정 행에 공백(스페이스 한 칸)이 있는 데이터 조회해서
# NaN(null)으로 변환한 뒤에
# null인 행 삭제 함수를 사용해서 삭제시키기
### 공백이 있는 행을 찾기
train_data["document"] = \
        train_data["document"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
train_data["document"].replace("", np.nan, inplace=True)

### 결측치가 있다면, 행 제거하기
train_data = train_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [68]:
### 단어 토큰화
# - 불용어 처리 동시에 진행
okt = Okt()

In [69]:
### 불용어 정의하기
stopwords = ["의", "가", "이", "은", "들", "는", "좀",
             "잘", "걍", "과", "도", "를", "으로", "자",
             "에", "와", "한", "하다"]
print(stopwords)

['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']


In [70]:
### 단어 토큰화
# - 훈련데이터
X_train = []
for sentence in tqdm(train_data["document"]) :
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem = True)
    # 불용어 처리
    stopwords_removed_sentence = [word
                                     for word in tokenized_sentence
                                         if not word in stopwords]
    # 각 문장별 단어 토큰화 결과 리스트에 추가하기
    X_train.append(stopwords_removed_sentence)
    
print(X_train)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 649.05it/s]

[['아', '더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'], ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'], ['사이', '몬페', '그', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '에서', '늙다', '보이다', '커스틴', '던스트', '너무나도', '이쁘다', '보이다'], ['막', '걸음', '마', '떼다', '세', '부터', '초등학교', '학년', '생인', '살다', '영화', 'ㅋㅋㅋ', '별', '반개', '아깝다', '움'], ['원작', '긴장감', '을', '제대로', '살리다'], ['별', '반개', '아깝다', '욕', '나오다', '이응경', '길용우', '연', '기', '생활', '몇', '년', '인지', '정말', '발', '로', '해도', '그것', '보단', '낫다', '납치', '감금', '만', '반복', '반복', '드라마', '가족', '없다', '연기', '못', '사람', '만', '모', '엿', '네'], ['액션', '없다', '재미', '있다', '몇', '안되다', '영화'], ['왜케', '평점', '낮다', '꽤', '볼', '만', '데', '헐리우드', '식', '화려하다', '너무', '길들이다', '있다'], ['인피니트', '짱', '이다', '진짜', '짱', '이다'], ['볼때', '마다', '눈물나다', '죽다', '년대', '향수', '극', '허진호', '감성', '절제', '멜로', '달인', '이다'], ['울면', '서', '손', '고', '횡단보도', '건너다', '때', '뛰다', '치다', '올', '뻔', '이범수', '연기', '드럽다', '못'], ['담백하다', '깔끔하다

In [71]:
### 테스트 데이터 정제 작업 먼저 진행
# - 한글과 공백을 제외하고 모두 제거하기
test_data["document"] = \
    test_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
test_data

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...,...
95,6753658,전기톱은못들고다니는데 엔진톱이겠죠,0
96,9665771,완전 재밌엇는데 왜 평점이,1
97,8757576,제임스 완이 내 목표임 ㄷ,1
98,9850643,점고 아깝다 개막장 영화의 원조라고나 할까아내와 사별한 지 얼마나 지났다고 딴 여자...,0


In [72]:
### 특정 행에 공백(스페이스 한 칸)이 있는 데이터 조회해서
# NaN(null)으로 변환한 뒤에
# null인 행 삭제 함수를 사용해서 삭제시키기
### 공백이 있는 행을 찾기
test_data["document"] = \
        test_data["document"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
test_data["document"].replace("", np.nan, inplace=True)

### 결측치가 있다면, 행 제거하기
test_data = test_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
test_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [73]:
### 단어 토큰화
# - 테스트 데이터
## 공백이 있으면 안되므로 확인하기
X_test = []
for sentence in tqdm(test_data["document"]) :
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem = True)
    # 불용어 처리
    stopwords_removed_sentence = [word
                                     for word in tokenized_sentence
                                         if not word in stopwords]
    # 각 문장별 단어 토큰화 결과 리스트에 추가하기
    X_test.append(stopwords_removed_sentence)
    
print(X_test)

100%|█████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 435.98it/s]

[['굳다', 'ㅋ'], ['뭐', '야', '평점', '나쁘다', '않다', '점', '짜다', '리', '더', '더욱', '아니다'], ['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기', '에는'], ['만', '아니다', '별', '다섯', '개', '주다', '왜', '로', '나오다', '제', '심기', '불편하다'], ['음악', '주가', '되다', '최고', '음악', '영화'], ['진정하다', '쓰레기'], ['마치', '미국', '애니', '에서', '튀어나오다', '창의력', '없다', '로봇', '디자인', '부터가', '고개', '젖다'], ['갈수록', '개판', '되다', '중국영화', '유치하다', '내용', '없다', '폼', '잡다', '끝나다', '말', '안되다', '무기', '유치하다', '남무', '아', '그리다', '동사서독', '같다', '영화', '이건', '류', '아', '류작', '이다'], ['이별', '아픔', '뒤', '찾아오다', '새롭다', '인연', '기쁨', '모든', '사람', '그렇다', '않다'], ['괜찮다', '오랜', '만', '포켓몬스터', '잼밌', '어', '요'], ['한국', '독립영화', '한계', '그렇게', '아버지', '되다', '비교', '되다'], ['청춘', '아름답다', '그', '아름답다', '움', '이성', '을', '흔들다', '놓다', '찰나', '아름답다', '움', '을', '자다', '포착', '섬세하다', '아름답다', '수채화', '같다', '퀴어', '영화', '이다'], ['눈', '보이다', '반전', '이다', '영화', '흡인', '력', '사라지다', '않다'], ['스토리', '연출', '연기', '비주', '얼', '등', '영화', '기본', '조차', '안되다', '영화', '무슨', '평', '을', '해', '이렇다', '영화', '찍다', '김문옥', '감독', '내', '영화', 

In [50]:
# 정수인코딩을 위한 객체 생성
tokenizer = Tokenizer()

In [51]:
# 정수인코딩하기 : 딕셔러니 형태로 만들어짐
# 빈도를 자동으로 계산해서 빈도가 높은 순서대로 1부터 값을 부여
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

AttributeError: 'int' object has no attribute 'lower'

In [ ]:
# 감성분석을 위해 단어 집합(각 문장)의 크기 확인 필요
# 단어 집합에서 희귀 단어수 확인 필요
# 전체 등장 빈도 대비 희귀 단어 등장 빈도 비율 확인 필요
# 희귀단어 정의 : 빈도가 몇 보다 작으면 희귀단어로 취급, 작다라는 기준은 분석가가 정의

In [52]:
# 희귀단어 기준 정의
threshold = 3
# 전체 단어 수
total_cnt = len(tokenizer.word_index)
# 희귀단어 기준보다 작은 단어의 개수
rare_cnt = 0
# 훈련 데이터의 전체 단어 빈도수 총합
total_freq = 0
# 희귀단어 기준보다 작은 단어의 빈도수 총합
rare_freq = 0

In [53]:
# 단어 빈도 개수 확인
tokenizer.word_counts

OrderedDict()

In [54]:
# 단어와 빈도수의 쌍을 key와 value로 받아서 반복 처리하기
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    # 희귀단어 기준보다 작으면
    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [55]:
print("단어집합의 크기 : ", total_cnt)
print("희귀 단어 기준보다 작은 단어수 : ", threshold - 1, rare_cnt)
print("단어 집합에서 희귀 단어의 비율 : ", rare_cnt / total_cnt * 100)
print("전체 등장 빈도에서 희귀 단어 등장 비율 : ", rare_freq / total_freq * 100)

단어집합의 크기 :  0
희귀 단어 기준보다 작은 단어수 :  2 0


ZeroDivisionError: division by zero

In [56]:
### 정수인코딩에 적용할 단어의 갯수(사이즈) 결정하기
vocab_size = total_cnt - rare_cnt + 1
print("사이즈 : ", vocab_size)

사이즈 :  1


In [57]:
### 텍스트 시퀀스(문자)를 정수 시퀀스(정수값)으로 변환
tokenizer = Tokenizer(vocab_size)

### 사이즈만큼의 단어들에 대해서 변환하기
tokenizer.fit_on_texts(X_train)

### 훈련 및 테스트 데이터 변환 적용하기
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(X_train,"\n")
print(X_test)

AttributeError: 'int' object has no attribute 'lower'

In [36]:
### 훈련 및 테스트 종속변수 생성하기
y_train = np.array(train_data["label"])
y_test = np.array(test_data["label"])
print(y_train, "\n")
print(y_test)

[0 1 0 0 1 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 0
 1 0 1 1 0 1 1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1
 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1] 

[1 0 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0
 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1]


In [38]:
### X_train 데이터에 비어있는 데이터 찾아서 삭제하기
# - 길이가 0인 인덱스 삭제하는 방법으로...

### 각 리스트의 길이가 0인 인덱스 찾기
drop_train = [index for index, sentence in enumerate(X_train)
                          if len(sentence) < 1]

### 찾아낸 인덱스 삭제하기
# - 훈련과 독립변수 및 종속변수 모두 동일한 위치의 인덱스 삭제
X_train = np.delete(X_train, drop_train, axis = 0)
y_train = np.delete(y_train, drop_train, axis = 0)

print(len(X_train), len(y_train))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

In [35]:
# 패딩크기 결정하기
count = 0
max_len = 10
for sentence in X_train:
    if (len(sentence) <= max_len):
        count = count + 1
print("전체 데이터 중 길이가 {} 이하인 비율 : {}".format(max_len, (count/len(max_len))*100))

TypeError: object of type 'int' has no len()